In [31]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

## Model

In [32]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [33]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

In [34]:
pd.read_csv("../generated/final/events_clicks_CTR_train.csv", nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1           8      1330329        638             1.0       2.0    1     7   
2           8      1330329        638             1.0       2.0    1     7   
3           8      1330329        638             1.0       2.0    1     7   
4        2657      1271490     185255             1.0       2.0    1     7   

    geo   ad_id  clicked  adsPerDisplay  clicksPerShows  
0  2765   95724        0              4        0.034785  
1  2765  175694        0              4        0.202327  
2  2765  280430        1              4        0.256853  
3  2765  329774        0              4        0.052426  
4  2765   70081        0              4        0.150145

In [35]:
usecols = ['display_id', 'ad_id', 'clicksPerShows', 'adsPerDisplay', 'platform', 'traffic_source', 'clicked']
features = ['clicksPerShows', 'clicksPerShowsM', 'p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']
#features = ['clicksPerShows', 'clicksPerShowsM']
#features = ['p1', 'p2', 'p3', 't1', 't2', 't3', 'adsPerDisplay']

In [36]:
pd.read_csv("../generated/final/test_index.csv", nrows=10).head()

index
0     21
1     22
2     23
3     24
4     25

## Preparation

In [38]:
train_nrows, test_nrows = None, None       # 650000, 210000

In [39]:
train_index = pd.read_csv("../generated/final/train_index.csv", nrows=train_nrows)['index'].ravel()
test_index = pd.read_csv("../generated/final/test_index.csv", nrows=test_nrows)['index'].ravel()

In [40]:
nrows = max(train_index[-1], test_index[-1])+1
nrows

87141731

In [41]:
train = pd.read_csv("../generated/final/events_clicks_CTR_train.csv", usecols = usecols, nrows=nrows)
train.count()

display_id        87141731
traffic_source    87141731
platform          87141731
ad_id             87141731
clicked           87141731
adsPerDisplay     87141731
clicksPerShows    87141731
dtype: int64

In [42]:
test = train.ix[test_index].copy()
train = train.ix[train_index].copy()

In [43]:
platform_oneshot = pd.DataFrame({'platform':[1,2,3], 'p1':[1,0,0], 'p2':[0,1,0], 'p3':[0,0,1]})
platform_oneshot.head()

p1  p2  p3  platform
0   1   0   0         1
1   0   1   0         2
2   0   0   1         3

In [ ]:
traffic_oneshot = pd.DataFrame({'traffic_source':[1,2,3], 't1':[1,0,0], 't2':[0,1,0], 't3':[0,0,1]})
traffic_oneshot.head()

t1  t2  t3  traffic_source
0   1   0   0               1
1   0   1   0               2
2   0   0   1               3

In [ ]:
train = train.merge(platform_oneshot, on='platform').drop('platform', axis=1)
test = test.merge(platform_oneshot, on='platform').drop('platform', axis=1)

train = train.merge(traffic_oneshot, on='traffic_source').drop('traffic_source', axis=1)
test = test.merge(traffic_oneshot, on='traffic_source').drop('traffic_source', axis=1)

In [ ]:
scaler = preprocessing.StandardScaler()
excpt = ['clicksPerShowsM'] #, 'p1', 'p2', 'p3', 't1', 't2', 't3']
for f in features:
    if not f in excpt:
        print f
        scaler.fit(train[[f]])
        train[f] = scaler.transform(train[[f]]) 
        test[f] = scaler.transform(test[[f]])

In [ ]:
trainM = pd.DataFrame({'clicksPerShowsM':train.groupby('display_id')['clicksPerShows'].min()}).reset_index()
testM = pd.DataFrame({'clicksPerShowsM':test.groupby('display_id')['clicksPerShows'].min()}).reset_index()

In [ ]:
train = train.merge(trainM, on='display_id')
train.count()

In [ ]:
test = test.merge(testM, on='display_id')
test.count()

In [ ]:
train[features].head()

## Validation

In [ ]:
clf = linear_model.SGDClassifier(random_state=0, loss='log', fit_intercept=True, n_iter=5)
clf.fit(train[features], train['clicked'].ravel())

In [ ]:
clf.classes_

In [ ]:
clicks_test_sample = test[['display_id', 'ad_id', 'clicked']]
clicks_test_sample.count()

In [49]:
validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str}).sort_values('display_id')
validation.count()

display_id    39655
ad_id         39655
clicked       39655
dtype: int64

In [ ]:
test[features].head()

In [ ]:
test['res'] = pd.Series(clf.predict_proba(test[features])[:,1])
test.head()

In [ ]:
clf.coef_, clf.intercept_[0]

In [ ]:
prediction = test.sort_values(by=['display_id', 'res'], ascending=[True, False])
prediction.count()

In [ ]:
prediction=pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()
prediction.count()

In [46]:
prediction.head()

display_id                                              ad_id
0         120                                      153192 116987
1         197             78098 128496 300898 187797 65119 70943
2         796          143302 158632 202812 145293 285991 235440
3        1132  155232 149541 234445 67305 85397 150083 328479...
4        1681                          258183 98740 86836 145293

In [47]:
validation.head()

display_id   ad_id  clicked
191211         120  116987        1
43720          197   78098        1
190811         796  202812        1
204839        1132  150083        1
43075         1681  258183        1

In [48]:
print 'clicksPerShows :', model_evaluation(validation, prediction)  #0.662869594202      0.588499968978      

clicksPerShows : 0.588499968978
